# Method 4

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nn
import time

In [45]:
def word_to_vector(word: str, substrings: list) -> np.array:
    array = np.zeros(len(substrings))
    lst = []
    for i, alph in enumerate(substrings):
        lst.append(word.count(alph))
    return np.array(lst)

CZ_ALPHABET = ['a', 'á', 'b', 'c', 'č', 'd', 'ď',
               'e', 'é', 'ě', 'f', 'g', 'h', 'ch',
               'i', 'í', 'j', 'k', 'l', 'm', 'n',
               'ň', 'o', 'ó', 'p', 'q', 'r', 'ř',
               's', 'š', 't', 'ť', 'u', 'ú','ů',
               'v', 'w', 'x', 'y', 'ý','z', 'ž', ' ']

SUBSTRINGS = CZ_ALPHABET

In [46]:
data = pd.read_csv('data.csv', encoding = 'ansi', usecols=['Obec', 'Kraj'])
data = data[data['Obec'] != 'Praha']

data['Obec'] = data['Obec'].str.lower()
data['food'] = data['Obec'].apply(lambda x: word_to_vector(x, SUBSTRINGS))
data['food'] = data['food'].apply(lambda x: x / np.linalg.norm(x))

LIST_KRAJE = np.sort(data['Kraj'].unique().tolist())
data['expected'] = data['Kraj'].apply(lambda x: word_to_vector(x, LIST_KRAJE))

In [75]:
NETWORK_SHAPE = (len(SUBSTRINGS), 25, len(LIST_KRAJE))
ACT_FUNCS = [nn.sigmoid, nn.sigmoid]
ETA = 0.01

neural_network = nn.NeuralNetwork(shape=NETWORK_SHAPE, act_funcs=ACT_FUNCS, eta=ETA)

N_EPOCHS = 2

costs = neural_network.train(df=data[['food', 'expected']], n_epochs=N_EPOCHS)

AttributeError: 'NeuralNetwork' object has no attribute 'train'

In [49]:
np.argmin(_)

1